In [1]:
import librosa
import numpy as np
import os
from keras.models import load_model
from sklearn.preprocessing import StandardScaler
from keras.utils import np_utils
from keras import models
from keras.layers import Dense, Dropout
from keras.models import Sequential   # 网络模型
from keras.layers import Dense,Dropout

# python librosa库有时会出现bug，现实backend error，此时需要打开ffmpeg包，添加到环境变量，然后在librosa下的aread.py里打开,然后将ffmpeg.py的路径替换。

# 将训练数据先进行预处理，对每个曲目进行频谱分析，将各个特征的结果保存在array

In [2]:
classes = 'piano guitar violin changdi shoufengqin'.split()
data_set = []
label_set = [] 
label2id = {class1:i for i,class1 in enumerate(classes)}
id2label = {i:class1 for i,class1 in enumerate(classes)}
print(label2id)
for c in classes:
    print(c)
    for filename in os.listdir(f'D:/张斯然文件/musicdate/date/classes/{c}/'):
        songname = f'D:/张斯然文件/musicdate/date/classes/{c}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rms(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
 
        to_append = f'{np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'     
        for e in mfcc:
            to_append += f' {np.mean(e)}' 
        data_set.append([float(i) for i in to_append.split(" ")])
        label_set.append(label2id[c])


{'piano': 0, 'guitar': 1, 'violin': 2, 'changdi': 3, 'shoufengqin': 4}
piano


C:\Users\cm\anaconda3\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
C:\Users\cm\anaconda3\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


guitar
violin
changdi
shoufengqin


In [3]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data_set, dtype = float))
y = np_utils.to_categorical(np.array(label_set))
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print(X_train.shape[1])

26


# 准备模型的建立

In [5]:
def Model_creat():
    model= Sequential()
    model.add(Dense(units=120,activation='relu',input_shape=(X_train.shape[1],)))#构建两个层
    model.add(Dense(units=60,activation='relu'))
    model.add(Dense(units=30, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=5,activation='softmax'))#输出节点
    return model

In [6]:
model=Model_creat()

In [7]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy','categorical_accuracy'])

### 模型的训练过程还是出现过许多问题，有时候在最后一层使用relu，使得loss不能成功计算，通过不停的调整参数和进行比较，得到比较好的结果有model13,modelshow,均已经保存在savemodel里了

In [8]:
model.fit(X_train, y_train, epochs=50, batch_size=64)

Epoch 1/50
2/2 [==============================] - 17s 3ms/step - loss: 1.7172 - accuracy: 0.1721 - categorical_accuracy: 0.1721
Epoch 2/50
2/2 [==============================] - 0s 5ms/step - loss: 1.5585 - accuracy: 0.2248 - categorical_accuracy: 0.2248
Epoch 3/50
2/2 [==============================] - 0s 4ms/step - loss: 1.4352 - accuracy: 0.2900 - categorical_accuracy: 0.2900
Epoch 4/50
2/2 [==============================] - 0s 2ms/step - loss: 1.3507 - accuracy: 0.3613 - categorical_accuracy: 0.3613
Epoch 5/50
2/2 [==============================] - 0s 2ms/step - loss: 1.2508 - accuracy: 0.4021 - categorical_accuracy: 0.4021
Epoch 6/50
2/2 [==============================] - 0s 2ms/step - loss: 1.1750 - accuracy: 0.5319 - categorical_accuracy: 0.5319
Epoch 7/50
2/2 [==============================] - 0s 2ms/step - loss: 1.1046 - accuracy: 0.6031 - categorical_accuracy: 0.6031
Epoch 8/50
2/2 [==============================] - 0s 2ms/step - loss: 1.1132 - accuracy: 0.6231 - categorical_

# 保存训练好的模型

In [1]:

#model.save(f'D:/张斯然文件/musicdate/date/modelshow.h5')

# 尝试进行验证，选了训练集中没有的5个曲目，保存在数据集目录下classes下的test类里，结果发现还挺准的，但是看概率的话，有一两个可能有点模糊

c——长笛

g——吉他

s——手风琴

p——钢琴

v——小提琴

In [9]:
import librosa
import numpy as np
import os
from keras.models import load_model
from sklearn.preprocessing import StandardScaler
from keras.utils import np_utils
model_path = f'D:/张斯然文件/musicdate/date/model13.h5'
# 载入模型
model = load_model(model_path)
data_set1=[]
for filename in os.listdir(f'D:/张斯然文件/musicdate/date/classes/test/'):
    songname = f'D:/张斯然文件/musicdate/date/classes/test/{filename}'
    print(songname)
    y, sr = librosa.load(songname, mono=True, duration=30)
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
    rmse = librosa.feature.rms(y=y)
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    zcr = librosa.feature.zero_crossing_rate(y)
    mfcc = librosa.feature.mfcc(y=y, sr=sr)
 
    to_append = f'{np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
 
    for e in mfcc:
        to_append += f' {np.mean(e)}'
 
    data_set1.append([float(i) for i in to_append.split(" ")])
        
scaler = StandardScaler()    
X1= scaler.fit_transform(np.array(data_set1, dtype = float))
result=model.predict(X1)
array=np.argmax(result,axis=1)
print(result)
print(array)
classes=['钢琴','吉他','小提琴','长笛','手风琴']
for j in range(0,len(array)):
    print("第{}个为".format(j+1),classes[array[j]])

D:/张斯然文件/musicdate/date/classes/test/c1.mp3


C:\Users\cm\anaconda3\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


D:/张斯然文件/musicdate/date/classes/test/g1.mp3
D:/张斯然文件/musicdate/date/classes/test/p1.mp3
D:/张斯然文件/musicdate/date/classes/test/s1.mp3
D:/张斯然文件/musicdate/date/classes/test/v1.mp3
[[1.84158352e-03 1.42207171e-03 1.31978682e-04 9.96122897e-01
  4.81391791e-04]
 [2.12312430e-01 7.36021459e-01 5.59710013e-03 1.47150655e-03
  4.45974581e-02]
 [8.45506310e-01 1.22267537e-01 5.69243683e-03 1.21670775e-02
  1.43666435e-02]
 [3.53744836e-03 2.76785381e-02 3.68632287e-01 1.07908566e-02
  5.89360833e-01]
 [3.21190059e-02 6.64945394e-02 6.67403102e-01 4.00314108e-03
  2.29980201e-01]]
[3 1 0 4 2]
第1个为 长笛
第2个为 吉他
第3个为 钢琴
第4个为 手风琴
第5个为 小提琴
